In [1]:
import copy
import os
import random
import sys

In [2]:
import numpy as np
from tqdm import trange

In [3]:
import time

In [4]:
from build_tree import (build_center_single, build_distribute_four,
                        build_distribute_nine,
                        build_in_center_single_out_center_single,
                        build_in_distribute_four_out_center_single,
                        build_left_center_single_right_center_single,
                        build_up_center_single_down_center_single)
from sampling import sample_attr, sample_attr_avail, sample_rules
from Rule import Rule_Wrapper

In [5]:
from rendering import render_panel

## Description
### 순서

1. Tree 생성
2. Define Rule

Rule적용한 후 결과물 생성
0. Rendering


# 0. Rendering
 미리 선언해둔 개체를 규칙에 맞게 numpy.array 위의 픽셀 파일로 변환해주는 과정
 
 이후 np.array를 cv2.imshow() 를 활용해 새 창에서 프린트 한다 (Only in 로컬 환경)
 
 rendering.py 의 함수들을 보면 계층 별로 그릴수 있는 함수 존재

In [6]:
import cv2

In [75]:
img = render_panel(row_1_1)

In [76]:
# turn on img with new window
cv2.imshow('result', img)
cv2.waitKey(1)

-1

In [77]:
img1 = render_panel(start_node)

In [78]:
# turn on img with new window
cv2.imshow('result1', img1)
cv2.waitKey(1)

-1

In [107]:
# turn off all windows
cv2.destroyAllWindows()

# 1. Tree 생성

In [11]:
class args():
    pass

In [12]:
args.num_samples = 5

In [13]:
all_configs = {"center_single": build_center_single(),
                   "distribute_four": build_distribute_four(),
                   "distribute_nine": build_distribute_nine(),
                   "left_center_single_right_center_single": build_left_center_single_right_center_single(),
                   "up_center_single_down_center_single": build_up_center_single_down_center_single(),
                   "in_center_single_out_center_single": build_in_center_single_out_center_single(),
                   "in_distribute_four_out_center_single": build_in_distribute_four_out_center_single()}

In [14]:
# for key in all_configs.keys():
key = "distribute_four"

In [15]:
for k in trange(args.num_samples):
    cont_num = k % 10

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<?, ?it/s]


In [16]:
root = all_configs[key]

선택한 config에 따라 적용가능한 Rule Group을 탐색한다

Rule을 샘플한 이후 Root을 이에 따라 Pruning 해보고 불가능하다 판단하면 에러발생 => 재시도 몇번해보면 에러 없이 적합한 root과 rule 조합을 반환함

In [17]:
while True:
    rule_groups = sample_rules()
    new_root = root.prune(rule_groups)
    if new_root is not None:
        break

In [18]:
start_node = new_root.sample()

# 계층구조 탐색
각 계층의 하위구조를 .children을 활용해 호출가능하며 복수의 하위구조를 가졌을 경우 Numbering을 통해 호출

대부분의 Operation은 Component 아래 계층 class의 attr를 바꾸는 형식으로 구현할 수 있을 것 같습니다

## Entity attr 요약
- .color : 해당 개체가 가질 color 값
- .bbox : 해당 개체가 그려질 위치 표현 
- .type : 해당 개체의 이름 (ex: 'triangle' )

In [19]:
row_1_1 = copy.deepcopy(start_node)
row_1_1.children[0].children[0].children[0].children

[Entity.0, Entity.1, Entity.2]

In [52]:
second_layout = row_1_1.children[0].children[0].children[0]
second_layout

Layout.Distribute_Four

**두 개체의 위치 Swap**

이거는 아직 함수 구현이 안 되어 있고 다음과 같은 형태로 간단히 구현할 수 있을 것 같다는걸 보이는 내용입니다


In [71]:
second_layout.children[0].bbox, second_layout.children[1].bbox = second_layout.children[1].bbox, second_layout.children[0].bbox

## 2. Define Rules
 우리가 원하는 정확한 Rule을 만들기 위한 과정
### Rule_Wrapper(name, attr, param, component_idx, entity_idxs)
- name : Rule.py에 존재하는 클래스 중 하나 선택
- attr : Rule이 변화시킬 대상 선택(Type / Size / Color)
- param : 지정한 attr이 변화할 폭 선택 (list)
- component_idx : 변화시킬 component 선택
- entity_idxs : 해당 component에서 변화할 entity의 위치 선택

In [33]:
rule = Rule_Wrapper(name = 'Progression_new', 
             attr = 'Size',
             param = [1],
             component_idx = 0,
             entity_idxs = (1, 2))

이후 rule class의 apply_rule 함수를 활용해 특정 대상에 적용시킨다

이제 output을 Rendering 시키면 결과를 확인할 수 있음

In [35]:
output = rule.apply_rule(row_1_1).

## 2-1. Rule을 적용할 target 찾기 (필수아님)

### set_target_with_condition(AoT, attr, value)
- AoT : target을 찾을 AoT 노드 대상 입력
- attr, value : 해당 attr의 value 값을 가진 Entuty의 idx를 반환한다
- results = { 'component_idx' : [Entity_idxs ] } 형태

In [105]:
def set_target_with_condition(AoT, attr, value):
    results = {}
    Components = AoT.children[0].children
    for comp_idx, comp in enumerate(Components):
        Entities = comp.children[0].children
        entity_idxs = []
        for enti_idx, enti in enumerate(Entities):
            if attr == "Type":
                if enti.type.get_value() == value:
                    entity_idxs += [enti_idx]
        results[comp_idx] = entity_idxs
        
    return results

In [106]:
set_target_with_condition(start_node, "Type", "triangle")

{0: [1]}